In [11]:

import datetime as dt
import pandas as pd
import pyspark
import pyspark.pandas as ps
from pyspark.sql import SparkSession

In [12]:
# oil diesel by UF and type: $DPCache_m3_2
diesel_df = pd.read_excel('../instance/vendas_teste.ods', 'DPCache_m3_2')

In [13]:
# oil derivative fuels by UF and product: $DPCache_m3
oilder_df = pd.read_excel('../instance/vendas_teste.ods', 'DPCache_m3')

In [4]:
unit = 'm3'

In [5]:
set(diesel_df['COMBUSTÍVEL'].to_list())

{'ÓLEO DIESEL (OUTROS ) (m3)',
 'ÓLEO DIESEL MARÍTIMO (m3)',
 'ÓLEO DIESEL S-10 (m3)',
 'ÓLEO DIESEL S-1800 (m3)',
 'ÓLEO DIESEL S-500 (m3)'}

In [28]:
diesel_pd_df = diesel_df.toPandas()
oilder_pd_df = oilder_df.toPandas()

In [6]:
months = ['Jan', 'Fev', 'Mar', 'Abr', 'Mai', 'Jun', 'Jul', 'Ago', 'Set', 'Out', 'Nov', 'Dez']
months_to_int_d = {m: i for m, i in zip(months, range(1, 12+1))}
months_to_int = lambda x: months_to_int_d[x]

In [7]:
months_to_int('Jan')

1

In [8]:
diesel_df

,COMBUSTÍVEL,ANO,REGIÃO,ESTADO,Jan,Fev,Mar,Abr,Mai,Jun,Jul,Ago,Set,Out,Nov,Dez,TOTAL
0,ÓLEO DIESEL S-10 (m3),2013,REGIÃO NORTE,RONDÔNIA,81453.67,3517.600,3681.700,4700.670,5339.200,6166.4,6539.65,7283.7,8082.85,7902.550,9383.150,9767.40,9088.800
1,ÓLEO DIESEL S-10 (m3),2013,REGIÃO NORTE,ACRE,1483.00,11202.000,363.000,410.000,536.000,607.0,740.00,756.0,971.00,1174.000,1240.000,1439.00,1483.000
2,ÓLEO DIESEL S-10 (m3),2013,REGIÃO NORTE,AMAZONAS,6836.30,6784.232,61443.832,3190.585,3305.000,3391.0,3637.00,4250.0,4576.00,5756.879,6228.636,6334.00,7154.200
3,ÓLEO DIESEL S-10 (m3),2013,REGIÃO NORTE,RORAIMA,1475.30,1502.700,1531.800,13423.700,795.400,757.2,939.80,1040.6,966.00,992.900,1027.000,1083.80,1311.200
4,ÓLEO DIESEL S-10 (m3),2013,REGIÃO NORTE,PARÁ,40913.48,45383.500,44013.219,41975.030,441140.785,30137.8,28146.30,31280.5,33033.05,33519.880,34321.530,37168.16,41248.336
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1075,ÓLEO DIESEL (OUTROS ) (m3),2020,REGIÃO SUL,RIO GRANDE DO SUL,0.00,0.000,10.000,10.000,0.000,10.0,NaN,NaN,NaN,60.000,10.000,10.00,10.000
1076,ÓLEO DIESEL (OUTROS ) (m3),2020,REGIÃO CENTRO-OESTE,MATO GROSSO DO SUL,0.00,0.000,0.000,0.000,0.000,0.0,0.00,NaN,NaN,NaN,0.000,0.00,0.000
1077,ÓLEO DIESEL (OUTROS ) (m3),2020,REGIÃO CENTRO-OESTE,MATO GROSSO,45.00,0.000,0.000,0.000,0.000,0.0,0.00,0.0,NaN,NaN,NaN,90.00,45.000
1078,ÓLEO DIESEL (OUTROS ) (m3),2020,REGIÃO CENTRO-OESTE,GOIÁS,0.00,0.000,0.000,0.000,0.000,0.0,0.00,0.0,41.40,NaN,NaN,NaN,41.400


In [9]:
diesel_df = diesel_df.melt(id_vars=['ANO', 'COMBUSTÍVEL', 'ESTADO'], value_vars=months, var_name='MONTH', value_name='volume')

## Schema

Data should be stored in the following format:

| Column       | Type        |
| ------------ | ----------- |
| `year_month` | `date`      |
| `uf`         | `string`    |
| `product`    | `string`    |
| `unit`       | `string`    |
| `volume`     | `double`    |
| `created_at` | `timestamp` |

In [10]:
diesel_df['month_int'] = diesel_df.apply(lambda x: months_to_int(x['MONTH']), axis=1)

In [11]:
diesel_df['year_month'] = diesel_df.apply(lambda x: dt.datetime(x['ANO'], x['month_int'], 1), axis=1)

In [12]:
diesel_df.rename({'COMBUSTÍVEL': 'product', 'ESTADO': 'uf'}, inplace=True, axis=1)

In [13]:
diesel_df['unit'] = 'm3'

In [14]:
diesel_df['created_at'] = pd.Timestamp.now()

In [15]:
diesel_final = diesel_df[['year_month', 'uf', 'product', 'unit', 'volume', 'created_at']]

In [16]:
diesel_final

,year_month,uf,product,unit,volume,created_at
0,2013-01-01,RONDÔNIA,ÓLEO DIESEL S-10 (m3),m3,81453.67,2023-03-30 06:02:56.588435
1,2013-01-01,ACRE,ÓLEO DIESEL S-10 (m3),m3,1483.00,2023-03-30 06:02:56.588435
2,2013-01-01,AMAZONAS,ÓLEO DIESEL S-10 (m3),m3,6836.30,2023-03-30 06:02:56.588435
3,2013-01-01,RORAIMA,ÓLEO DIESEL S-10 (m3),m3,1475.30,2023-03-30 06:02:56.588435
4,2013-01-01,PARÁ,ÓLEO DIESEL S-10 (m3),m3,40913.48,2023-03-30 06:02:56.588435
...,...,...,...,...,...,...
12955,2020-12-01,RIO GRANDE DO SUL,ÓLEO DIESEL (OUTROS ) (m3),m3,10.00,2023-03-30 06:02:56.588435
12956,2020-12-01,MATO GROSSO DO SUL,ÓLEO DIESEL (OUTROS ) (m3),m3,0.00,2023-03-30 06:02:56.588435
12957,2020-12-01,MATO GROSSO,ÓLEO DIESEL (OUTROS ) (m3),m3,90.00,2023-03-30 06:02:56.588435
12958,2020-12-01,GOIÁS,ÓLEO DIESEL (OUTROS ) (m3),m3,NaN,2023-03-30 06:02:56.588435


In [42]:
dt.datetime(1988, 5, 1)

datetime.datetime(1988, 5, 1, 0, 0)

In [52]:
diesel_melted.assign(year_month=lambda x: dt.datetime(x.ANO, months_to_int(x.variable), 1))

TypeError: unhashable type: 'Series'

In [45]:
diesel_melted['ANO']

0        2013
1        2013
2        2013
3        2013
4        2013
         ... 
12955    2020
12956    2020
12957    2020
12958    2020
12959    2020
Name: ANO, Length: 12960, dtype: int64

In [12]:
type(diesel_df)

pyspark.sql.dataframe.DataFrame

In [13]:
oilder_df.melt(
        ids=['ANO'], values=['Jan', 'Fev'],
        variableColumnName="variable",
        valueColumnName="value")

AttributeError: 'DataFrame' object has no attribute 'melt'

In [38]:
set(oilder_df.toPandas()['COMBUSTÍVEL'].to_list())

{'ETANOL HIDRATADO (m3)',
 'GASOLINA C (m3)',
 'GASOLINA DE AVIAÇÃO (m3)',
 'GLP (m3)',
 'QUEROSENE DE AVIAÇÃO (m3)',
 'QUEROSENE ILUMINANTE (m3)',
 'ÓLEO COMBUSTÍVEL (m3)',
 'ÓLEO DIESEL (m3)'}

In [25]:
oilder_df['ESTADO']

0                 RONDÔNIA
1                     ACRE
2                 AMAZONAS
3                  RORAIMA
4                     PARÁ
5                    AMAPÁ
6                TOCANTINS
7                 MARANHÃO
8                    PIAUÍ
9                    CEARÁ
10     RIO GRANDE DO NORTE
11                 PARAÍBA
12              PERNAMBUCO
13                 ALAGOAS
14                 SERGIPE
15                   BAHIA
16            MINAS GERAIS
17          ESPÍRITO SANTO
18          RIO DE JANEIRO
19               SÃO PAULO
20                  PARANÁ
21          SANTA CATARINA
22       RIO GRANDE DO SUL
23      MATO GROSSO DO SUL
24             MATO GROSSO
25                   GOIÁS
26        DISTRITO FEDERAL
27                RONDÔNIA
28                    ACRE
29                AMAZONAS
30                 RORAIMA
31                    PARÁ
32                   AMAPÁ
33               TOCANTINS
34                MARANHÃO
35                   PIAUÍ
36                   CEARÁ
3

In [19]:
# Prepare Data
data = [("Spain", 101, 201, 301),
        ("Taiwan", 102, 202, 302),
        ("Italy", 103, 203, 303),
        ("China", 104, 204, 304)
  ]

# Create DataFrame
columns= ["Country", "2018", "2019", "2020"]
df = spark.createDataFrame(data = data, schema = columns)
df.show(truncate=False)

+-------+----+----+----+
|Country|2018|2019|2020|
+-------+----+----+----+
|Spain  |101 |201 |301 |
|Taiwan |102 |202 |302 |
|Italy  |103 |203 |303 |
|China  |104 |204 |304 |
+-------+----+----+----+



In [ ]:
from pyspark.sql.functions import col, expr, concat, lit, from_unixtime, unix_timestamp


In [25]:
import pyspark.sql.functions as F
import itertools

months = ['Jan', 'Fev', 'Mar', 'Abr', 'Mai', 'Jun', 'Jul', 'Ago', 'Set', 'Out', 'Nov', 'Dez']

df = oilder_df.withColumn("year_gain_map", F.create_map(list(itertools.chain(months)))) \
       .select("Ano", F.explode("year_gain_map").alias("Year", "CPI"))

In [26]:
df.show()

+----+----------+---------+
| Ano|      Year|      CPI|
+----+----------+---------+
|2000|136073.253| 9563.263|
|2000| 11341.229| 9369.746|
|2000| 10719.983|11165.968|
|2000| 12312.451| 11220.97|
|2000| 12482.281|13591.122|
|2000|  11940.57|11547.576|
|2000|  3358.346|40001.853|
|2000|  3065.758|  3495.29|
|2000|   2946.93|  3023.92|
|2000|   3206.93|  3612.58|
|2000|   3264.46|  3835.74|
|2000|  3676.571|  3225.61|
|2000| 20766.918|21180.919|
|2000|242742.352|17615.604|
|2000|   20258.2|18741.344|
|2000| 19604.023|20221.674|
|2000| 20792.616|19912.898|
|2000| 21869.338|21145.643|
|2000|  3716.032|   3200.4|
|2000|  3339.332|43338.929|
+----+----------+---------+
only showing top 20 rows

